# Data Warehouse Design

## Overview
In this notebook, we will learn the fundamentals of **Data Warehousing** including:

- Concepts and architecture
- Star Schema and Snowflake Schema
- Fact and Dimension tables
- ETL pipelines for populating data warehouses
- Hands-on design and querying using SQLite (or any RDBMS)


## What is a Data Warehouse?

A **Data Warehouse (DW)** is a central repository of integrated data from multiple sources.
It supports **business intelligence (BI)** and **analytics**, enabling faster decision-making.

### Key Features:
- Subject-oriented: Organized by business domain (e.g., sales, customers)
- Integrated: Data comes from multiple heterogeneous sources
- Time-variant: Stores historical data for trend analysis
- Non-volatile: Data is read-only after loading

## Architecture of a Data Warehouse
1. **Data Sources**
   - Operational databases (OLTP systems)
   - Flat files, APIs, external datasets
2. **ETL Layer**
   - **Extract:** Pull data from sources
   - **Transform:** Clean, normalize, enrich data
   - **Load:** Insert into the Data Warehouse
3. **Data Storage Layer**
   - Fact tables: Store measurable, quantitative data
   - Dimension tables: Store descriptive attributes
4. **Analytics & BI Tools**
   - Querying: SQL, OLAP
   - Visualization: Tableau, Power BI
## Fact vs Dimension Tables

### Fact Table
- Contains **quantitative data** (metrics, measurements)
- Example: `Sales_Fact` table with `Sales_Amount`, `Quantity`, `Revenue`

### Dimension Table
- Contains **descriptive attributes** (dimensions for analysis)
- Example: `Customer_Dim` table with `Customer_Name`, `Region`, `Age`

### Relationship
- Fact tables usually have **foreign keys** pointing to dimension tables
- Enables **star schema** or **snowflake schema** design
## Star Schema vs Snowflake Schema

### Star Schema
- Single fact table linked to multiple dimension tables
- Simple, fast queries
- Denormalized dimensions

### Snowflake Schema
- Dimension tables are **normalized** into multiple related tables
- Reduces redundancy, but queries are more complex

## Hands-On: Data Warehouse Design using SQLite

We will create a **star schema** for a sample sales data warehouse.

- Fact Table: `Sales_Fact`
- Dimension Tables: `Customer_Dim`, `Product_Dim`, `Time_Dim`, `Store_Dim`


In [4]:
import sqlite3
import pandas as pd

In [5]:
# Create a SQLite database in memory
conn = sqlite3.connect("data_warehouse.db")
cursor = conn.cursor()


In [6]:
# Customer Dimension Table
cursor.execute("""
CREATE TABLE Customer_Dim (
    Customer_ID INTEGER PRIMARY KEY,
    Customer_Name TEXT,
    Age INTEGER,
    Gender TEXT,
    Region TEXT
)
""")

# Product Dimension Table
cursor.execute("""
CREATE TABLE Product_Dim (
    Product_ID INTEGER PRIMARY KEY,
    Product_Name TEXT,
    Category TEXT,
    Brand TEXT
)
""")

# Time Dimension Table
cursor.execute("""
CREATE TABLE Time_Dim (
    Time_ID INTEGER PRIMARY KEY,
    Date TEXT,
    Month TEXT,
    Quarter TEXT,
    Year INTEGER
)
""")

# Store Dimension Table
cursor.execute("""
CREATE TABLE Store_Dim (
    Store_ID INTEGER PRIMARY KEY,
    Store_Name TEXT,
    City TEXT,
    State TEXT,
    Country TEXT
)
""")
conn.commit()


In [7]:
# Fact Table: Sales_Fact
cursor.execute("""
CREATE TABLE Sales_Fact (
    Sales_ID INTEGER PRIMARY KEY,
    Customer_ID INTEGER,
    Product_ID INTEGER,
    Time_ID INTEGER,
    Store_ID INTEGER,
    Quantity INTEGER,
    Sales_Amount REAL,
    FOREIGN KEY(Customer_ID) REFERENCES Customer_Dim(Customer_ID),
    FOREIGN KEY(Product_ID) REFERENCES Product_Dim(Product_ID),
    FOREIGN KEY(Time_ID) REFERENCES Time_Dim(Time_ID),
    FOREIGN KEY(Store_ID) REFERENCES Store_Dim(Store_ID)
)
""")
conn.commit()


## Insert Sample Data

We will insert a few sample records into each dimension table.

In [8]:
# Customer Dimension
cursor.executemany("""
INSERT INTO Customer_Dim (Customer_ID, Customer_Name, Age, Gender, Region)
VALUES (?, ?, ?, ?, ?)
""", [
    (1, 'Alice', 30, 'F', 'North'),
    (2, 'Bob', 35, 'M', 'South'),
    (3, 'Charlie', 28, 'M', 'East')
])

# Product Dimension
cursor.executemany("""
INSERT INTO Product_Dim (Product_ID, Product_Name, Category, Brand)
VALUES (?, ?, ?, ?)
""", [
    (1, 'Laptop', 'Electronics', 'Dell'),
    (2, 'Phone', 'Electronics', 'Apple'),
    (3, 'Desk', 'Furniture', 'Ikea')
])

# Time Dimension
cursor.executemany("""
INSERT INTO Time_Dim (Time_ID, Date, Month, Quarter, Year)
VALUES (?, ?, ?, ?, ?)
""", [
    (1, '2025-10-01', 'October', 'Q4', 2025),
    (2, '2025-10-02', 'October', 'Q4', 2025),
    (3, '2025-10-03', 'October', 'Q4', 2025)
])

# Store Dimension
cursor.executemany("""
INSERT INTO Store_Dim (Store_ID, Store_Name, City, State, Country)
VALUES (?, ?, ?, ?, ?)
""", [
    (1, 'Store A', 'New York', 'NY', 'USA'),
    (2, 'Store B', 'Los Angeles', 'CA', 'USA')
])

conn.commit()


In [9]:
cursor.executemany("""
INSERT INTO Sales_Fact (Sales_ID, Customer_ID, Product_ID, Time_ID, Store_ID, Quantity, Sales_Amount)
VALUES (?, ?, ?, ?, ?, ?, ?)
""", [
    (1, 1, 1, 1, 1, 2, 2000.0),
    (2, 2, 2, 2, 2, 1, 1200.0),
    (3, 3, 3, 3, 1, 5, 500.0)
])
conn.commit()


## Query Examples

- Join fact and dimension tables to analyze sales
- Calculate total sales by product, customer, store, or time


In [10]:
# Total sales by Product
query = """
SELECT P.Product_Name, SUM(F.Sales_Amount) AS Total_Sales
FROM Sales_Fact F
JOIN Product_Dim P ON F.Product_ID = P.Product_ID
GROUP BY P.Product_Name
"""
df = pd.read_sql(query, conn)
df


,Product_Name,Total_Sales
0,Desk,500.0
1,Laptop,2000.0
2,Phone,1200.0


In [11]:
# Sales by Customer
query = """
SELECT C.Customer_Name, SUM(F.Sales_Amount) AS Total_Sales
FROM Sales_Fact F
JOIN Customer_Dim C ON F.Customer_ID = C.Customer_ID
GROUP BY C.Customer_Name
"""
df = pd.read_sql(query, conn)
df


,Customer_Name,Total_Sales
0,Alice,2000.0
1,Bob,1200.0
2,Charlie,500.0


In [12]:
# Sales by Store and Month
query = """
SELECT S.Store_Name, T.Month, SUM(F.Sales_Amount) AS Total_Sales
FROM Sales_Fact F
JOIN Store_Dim S ON F.Store_ID = S.Store_ID
JOIN Time_Dim T ON F.Time_ID = T.Time_ID
GROUP BY S.Store_Name, T.Month
"""
df = pd.read_sql(query, conn)
df


,Store_Name,Month,Total_Sales
0,Store A,October,2500.0
1,Store B,October,1200.0


# ✅ Summary

In this notebook, we covered:

- Fundamentals of **Data Warehousing**
- **Star Schema design** with Fact and Dimension tables
- ETL concept for populating data warehouses
- Hands-on creation using **SQLite**
- Querying and aggregating sales data

This is a foundational step toward building **Data Engineering pipelines** for Machine Learning and Analytics.
